In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import json
from zipfile import ZipFile
from sklearn.model_selection import train_test_split

In [3]:
kaggle_dictionary=json.load(open('api.json'))

In [4]:
os.environ['KAGGLE_USERNAME']=kaggle_dictionary['username']
os.environ['KAGGLE_KEY']=kaggle_dictionary['key']

In [5]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
!ls


api.json  imdb-dataset-of-50k-movie-reviews.zip  sample_data


In [8]:
with ZipFile('imdb-dataset-of-50k-movie-reviews.zip','r') as zip:
    zip.extractall()

In [6]:
!ls

 api.json  'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   sample_data


In [2]:
df=pd.read_csv('IMDB Dataset.csv')

In [3]:
df.shape

(50000, 2)

In [4]:
df=df.sample(40000,random_state=2)

In [5]:
df['sentiment'].value_counts()

,count
sentiment,
negative,20037
positive,19963


In [6]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [7]:
df.duplicated().sum()

281

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
# preprocessing the review coloumn
import re
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [10]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
def preprocess_review(review):
    # Step 1: Lowercasing
    review = review.lower()

    # Step 2: Remove HTML tags
    review = BeautifulSoup(review, "html.parser").get_text()

    # Step 3: Remove special characters and punctuation
    review = re.sub(r'[^a-zA-Z0-9\s]', '', review)

    # Step 4: Tokenize the review
    tokens = word_tokenize(review)

    # Step 5: Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Step 6: Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join tokens back into a single string
    preprocessed_review = ' '.join(tokens)

    return preprocessed_review

In [12]:
df['review'] = df['review'].apply(preprocess_review)

<ipython-input-11-675351800cf2>:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review = BeautifulSoup(review, "html.parser").get_text()


In [18]:
df['review'][0]

'one reviewer mentioned watching 1 oz episode youll hooked right exactly happened methe first thing struck oz brutality unflinching scene violence set right word go trust show faint hearted timid show pull punch regard drug sex violence hardcore classic use wordit called oz nickname given oswald maximum security state penitentary focus mainly emerald city experimental section prison cell glass front face inwards privacy high agenda em city home manyaryans muslim gangsta latino christian italian irish moreso scuffle death stare dodgy dealing shady agreement never far awayi would say main appeal show due fact go show wouldnt dare forget pretty picture painted mainstream audience forget charm forget romanceoz doesnt mess around first episode ever saw struck nasty surreal couldnt say ready watched developed taste oz got accustomed high level graphic violence violence injustice crooked guard wholl sold nickel inmate wholl kill order get away well mannered middle class inmate turned prison b

In [13]:
X=df['review']
y=df['sentiment']

In [14]:
X

,review
23656,ron hall pull triple threat writes directs sta...
27442,first series brilliant easily one best horror ...
40162,loved movie bobbie phillips really fight alway...
8459,13 yr age saw greatly underappreciated film ad...
8051,coen brother truly outdone wonderful saga thre...
...,...
10807,much hated movie series follows least say zang...
13170,movie fantastic great movie scary hell mean fr...
6319,yes ashamed admit show positively devineits en...
1891,opening scene notice hard tried mimic smart po...


In [27]:
y

,sentiment
23656,negative
27442,negative
40162,positive
8459,positive
8051,positive
...,...
10807,negative
13170,negative
6319,positive
1891,negative


In [15]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y=le.fit_transform(y)

In [16]:
y

array([0, 0, 1, ..., 1, 0, 1])

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)

In [18]:
X_train.shape

(31775,)

In [26]:
X_train

,review
5111,know film mean european film bad film regarded...
41608,movie wasnt bad terrible watched actually felt...
45222,emergence quentin tarantino dubious influence ...
46322,general trelayne superbeing want play little g...
18127,im even going waste time describing bad movie ...
...,...
44891,norma shearer dazzle transformed frump addlebr...
30434,still photographer working europe summer jim s...
6642,first anyone think sequel william friedkins fr...
35550,fame one best movie ive seen performing art mu...


In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)
X_train_cv=cv.fit_transform(X_train).toarray()
X_test_cv=cv.transform(X_test).toarray()

In [21]:
X_train_cv.shape

(31775, 5000)

In [22]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
nb.fit(X_train_cv,y_train)

MultinomialNB()

In [23]:
y_pred=nb.predict(X_test_cv)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8423967774420946

In [24]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train_cv,y_train)

RandomForestClassifier()

In [25]:
y_pred=rf.predict(X_test_cv)
accuracy_score(y_test,y_pred)

0.8377391742195368

In [29]:
# Example unseen data
unseen_data = ["I was really disappointed with this film. The plot was slow and predictable, and the characters felt one-dimensional. The dialogue was flat and lacked depth, making it hard to connect with the story. Even the special effects, which I was hoping would be a redeeming feature, felt outdated and unimpressive. The pacing was off, with long stretches of nothing happening, and the ending left much to be desired. Overall, it felt like a waste of time, and I wouldn’t recommend it to anyone looking for an engaging or entertaining movie experience."]


unseen_data_preprocessed = [preprocess_review(review) for review in unseen_data]


unseen_data_transformed = cv.transform(unseen_data_preprocessed)


predictions = nb.predict(unseen_data_transformed)


print(predictions)


[0]
